In [59]:
x1 = Entrez.esearch("Gene","Smad4")

In [60]:
ids = Entrez.read(x1)
ids

{'Count': '4323', 'RetMax': '20', 'RetStart': '0', 'IdList': ['132502303', '7157', '132504550', '1956', '7124', '7422', '3569', '7040', '22059', '2099', '6774', '672', '3845', '673', '4318', '367', '207', '29126', '4790', '1499'], 'TranslationSet': [], 'TranslationStack': [{'Term': 'Smad4[All Fields]', 'Field': 'All Fields', 'Count': '4323', 'Explode': 'N'}, 'GROUP'], 'QueryTranslation': 'Smad4[All Fields]'}

In [63]:
ids["IdList"][0]

'132502303'

In [81]:
handle = Entrez.efetch(db="gene", id=ids["IdList"][0], rettype="gb", retmode="text")

URLError: <urlopen error [Errno 8] nodename nor servname provided, or not known>

In [ ]:
print(handle)

In [41]:
gget.info(info.iloc[0][0])

,ensembl_id,uniprot_id,pdb_id,ncbi_gene_id,species,assembly_name,primary_gene_name,ensembl_gene_name,synonyms,parent_gene,...,transcript_names,transcript_strands,transcript_starts,transcript_ends,all_exons,exon_starts,exon_ends,all_translations,translation_starts,translation_ends
ENSG00000141646,ENSG00000141646.16,Q13485,NaN,4089,homo_sapiens,GRCh38,SMAD4,SMAD4,"[DPC4, JIP, MADH4, MYHRS]",NaN,...,"[SMAD4-206, SMAD4-202, SMAD4-207, SMAD4-209, S...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[51028528, 51029614, 51030100, 51030209, 51030...","[51085017, 51085045, 51085042, 51085042, 51085...",NaN,NaN,NaN,NaN,NaN,NaN
